# Import dataset

In [ ]:
# ! pip install -q kaggle
# from google.colab import files
# files.upload()
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json

# ! kaggle datasets list
# ! pip install --upgrade --force-reinstall --no-deps kaggle

# !kaggle competitions download -c nlp-getting-started -p 'dataset'

In [ ]:
# cd dataset

In [ ]:
# ! unzip nlp-getting-started.zip

In [ ]:
import pandas as pd

train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')
sample_submission = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
sample_submission

# EDA

In [ ]:
train.info()

In [ ]:
train.isna().sum()

In [ ]:
test.info()

In [ ]:
test.isna().sum()

As we can see not every cell is filled. But what we are interested in is the text and target rows which are fully filled

# CountVectorizer

we vectorize text column of train dataset

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cntv = CountVectorizer()
vectxt_train = cntv.fit_transform(train['text'])

we do have 21637 unique strings in whole tweets bag

In [ ]:
vectxt_train.toarray().shape

In [ ]:
vecdf_train = pd.DataFrame(data = vectxt_train.toarray(), columns = cntv.get_feature_names())
vecdf_train

Let's see how our strings perform

In [ ]:
words_cnt = cntv.vocabulary_
sorted(words_cnt.items(), key=lambda x: x[1], reverse = True)  

Let's transform our test 'text' column. But be aware we are using transform only since we want to use same model on test set and not to create second model what wouldn't have any sens.

In [ ]:
vectxt_test = cntv.transform(test['text'])

In [ ]:
vectxt_test.toarray().shape

In [ ]:
vecdf_test = pd.DataFrame(data = vectxt_test.toarray(), columns = cntv.get_feature_names())
vecdf_test

# Naive Bayes vs Ridge Regression

Naive Bayes instance

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb_model = GaussianNB()

Let's check how our model perfoms first

In [ ]:
from sklearn import model_selection

scores = model_selection.cross_val_score(gnb_model, vectxt_train.toarray(), train["target"], cv=10, scoring="f1")
scores

Ridge regression instance

In [ ]:
from sklearn import linear_model

rc_model = linear_model.RidgeClassifier()

ridge regression performance

In [ ]:
from sklearn import model_selection

scores = model_selection.cross_val_score(rc_model, vectxt_train, train["target"], cv=10, scoring="f1")
scores

I will use ridge regression since it managed to predict up to 69% accurate

# Train

In [ ]:
rc_model.fit(vectxt_train, train['target'])

In [ ]:
predictions = rc_model.predict(vectxt_test)
predictions

# Submission

In [ ]:
sample_submission['target'] = predictions
sample_submission['target'].value_counts()

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('ss.csv', index = False)

In [ ]:
ss = pd.read_csv('ss.csv')
ss

# Curiosity check:)

In [ ]:
test['disaster_check'] = predictions
check = test[['text','disaster_check']]

let's create a list of random 5 indexes from test.

In [ ]:
from random import randint

rand_int = []

for i in range(5):
  rand_int.append(randint(0,len(test)))

rand_int

display tweets and anwsers 

In [ ]:
pd.set_option('display.max_colwidth', None) # to display full content

for i in rand_int:
  print(check.iloc[i,:])
  print('\n')

It seems that the algorithm was able to predict correctly all of above examples. Of course it is arguable whether e.g. second sentence is a disaster. I mean this event is something that is probably already known by either police or medics. I think better example of (unexpected) disaster is 3rd sentence about terrorism because it can be unknown yet by the officials. Also quite cool example is 4th sentence where, despite the fact user used 'crushed' word in a tweet, algorithm correctly labeled it as 0.